<a href="https://colab.research.google.com/github/Blackman9t/Advanced-Data-Science/blob/master/apache_mean_std_skewness_kurtosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### We shall study some statistical moments in this notebook. These are:-

1. Mean<br>
2. Standard Deviation<br>
3. Skewness<br>
4. Kurtosis

**Note:**

Esc + M changes a code cell to markdown cell<br>
Esc + Y changes the markdown cell to code cell.
    

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz

!pip install -q findspark
!pip install pyspark
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

     |████████████████████████████████| 217.8MB 61kB/s 
     |████████████████████████████████| 204kB 48.4MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=bab2d1b5713fea40943f880b82d979b9114bef024dfc1dabc49fe0965f13fa3e
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [2]:
from pyspark import SparkConf, SparkContext
try:
    conf = SparkConf().setMaster("local").setAppName("My App")
    sc = SparkContext(conf = conf)
    print('SparkContext Initialised Successfully!')
except Exception as e:
    print(e)

SparkContext Initialised Successfully!


In [3]:
sc

<SparkContext master=local appName=My App>

In [4]:
# Next we set up a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('My App').getOrCreate()
spark

**The 1st Statistical Moment...**

### Let's calculate the mean of a range of numbers

In [5]:
rdd = sc.parallelize(range(100))
n = float(rdd.count())
summ = rdd.sum()
mean = summ / float(n)
print(mean)

49.5


**The 2nd Statistical Moment...**

### Knowing the mean, let's calculate the standard deviation(std)

In [6]:
std = ((rdd.map(lambda x: (x - mean)**2).sum())/n)**0.5
std

28.86607004772212

### Let's increase the number range and see what happens to the mean and std

In [7]:
# First calculate the mean
rdd = sc.parallelize([49] * 100 + [100])
n = rdd.count()
summ = rdd.sum()
mean = summ / n
print(mean)

49.504950495049506


In [8]:
std = ((rdd.map(lambda x: (x - mean)**2).sum())/n)**0.5
std

5.0495049504950495

We can see that the mean stays the same relatvely, unlike the std which has changed to 5.0.<br>
Thus the mean is more resistant to effects of outliers in a data, although the median is the most outlier resistant measure of central tendency.

Note that **variance** is simply just the square of the _standard deviation._

What's the mean of the list below

In [9]:
x = [1,2,4,5,34,1,32,4,34,2,1,3]
x = sc.parallelize(x)
summ = x.sum()
print(summ)
mean = summ / float(x.count())
mean

123


10.25

What's the median of the list below

In [10]:
x = [1,2,4,5,34,1,32,4,34,2,1,3]
x = sorted(x)
print(x)
if len(x) % 2 == 0:
    print('Even List')
    upper_ind = len(x) // 2
    lower_ind = (len(x) // 2) - 1
    print(x[lower_ind])
    print(x[upper_ind])
    median = (x[upper_ind] + x[lower_ind]) / 2.0
    print(median)
else:
    print("Odd List")
    index = len(x) // 2
    median = x[index]
    print(median)

[1, 1, 1, 2, 2, 3, 4, 4, 5, 32, 34, 34]
Even List
3
4
3.5


What's the standard deviation of these numbers

In [0]:
x = [34,1,23,4,3,3,12,4,3,1]

In [12]:
# First calculate the mean
rdd = sc.parallelize(x)
n = rdd.count()
summ = rdd.sum()
mean = summ / float(n)
print(mean)

8.8


In [13]:
std = ((rdd.map(lambda j: (j - mean)**2).sum())/n)**0.5
std

10.562196741208714

**The 3rd Statistical moment...**

**Skewness:**

This basically tells us how asymmetrical the data is spread around the mean. if the histogram has a tail, we have a skew.<br>
If the skew is on the right, it's positively skewed and left is negatively skewed. skew tells us about the asymmetry but not the shape.<br>
Note that incase we have a long-tailed thin left skew and a short-tail thick right skew, they might cancel out and we optain a value close to zero for skewness.<br>
So it is always wise to plot your data as well as calculate statistical moments.
<img src='https://github.com/Blackman9t/Coursera_Capstone/blob/master/skewness.png?raw=true' width=800 height=800 alt='Skewness_formula'>

In [36]:
# Let's calculate the mean of a range of numbers
rdd = sc.parallelize(range(100))
n = rdd.count()
summ = rdd.sum()
mean = summ / float(n)
print(mean)

49.5


In [37]:
# Let's calculate the Standard-deviation, now that we know the mean
# first let's calculate the Variance, which is the square of the Standard-deviation

variance = (rdd.map(lambda x:pow(x-mean,2)).sum()/n)
variance

833.25

In [38]:
# Now to calculate the Standard-deviation, let's simply get the square-root of the variance
std = pow(variance, 0.5)
std

28.86607004772212

Now that we have the number of items(n), and the mean, the variance and standard deviation, let's calculate the skewness.

In [39]:
skewness00 = rdd.map(lambda x: pow(x - mean, 3) / pow(std, 3)).sum()
skewness00

6.217248937900877e-15

Now let's normalise the skewness of these values of numbers

In [40]:
skewness = (1/n) * skewness00
skewness

6.217248937900877e-17

**The 4th Statistical moment...**

**Kurtosis:**

This basically tells us something about the shape of our data when plotted using a histogram. It indicates the outlier content within the data.<br>
The higher the Kurtosuis measure is, the more outliers are present and the longer the tails in the distribution of the histogram are.<br>

The formula for calculating Kurtosis:
<img src='https://github.com/Blackman9t/Coursera_Capstone/blob/master/kurtosis.png?raw=true' width=800 height=800 alt='Kurtosis_formula'>

Let's go ahead and calculate the kurtosis of the same number range 0 - 100

In [19]:
kurtosis = (rdd.map(lambda x: pow(x-mean, 4) / pow(std, 4)).sum()) / n
kurtosis

1.7997599759976

Kurtosis tells us about the relative number of outliers in our dataset and it reports on the shape of the histogram.

**What is the kurtosis of the following list?**

In [0]:
list1 = [34,1,23,4,3,3,12,4,3,1]

In [21]:
# To find the kurtosis, we need to deduce the mean and STD

kdd = sc.parallelize(list1)
n = float(kdd.count())
summ = kdd.sum()
print(summ)
mean = summ / n
mean

88


8.8

In [22]:
# Now we find the std

variance = (kdd.map(lambda x: pow(x-mean,2)).sum()) / n
std = pow(variance, 0.5)
std

10.562196741208714

In [23]:
# Now to find kurtosis we simply do

kurtosis = (kdd.map(lambda x: pow(x-mean, 4) / pow(std, 4)).sum()) / n
kurtosis

3.663124005193276

### Quiz
What is the skewness of the following list?

34,1,23,4,3,3,12,4,3,1

Please enter at least three digits after the decimal

In [24]:
# Since the list is same as list1 above we simply do
skewness  = (kdd.map(lambda x: pow(x-mean, 3) / pow(std, 3)).sum()) / n
skewness

1.4549069329914

### Covariance, Correlation and Matrices

_Covariance_ and _Correlation_ tell us how two columns are interacting with each other.<br>
**Covariance** is the measure of dependency between two columns in a data set.<br>
If both columns totally correlate, the measure is +1. Incase the two columns are showing inverse dependency, the Covariance is -1.<br>
Incase the two columns show no interaction atall, the Covariance is 0. Covariance can be calculated very easily.<br>
See its'formula below:
<img src= 'https://github.com/Blackman9t/Coursera_Capstone/blob/master/covariance.png?raw=true' width=800 height=800 alt='Covariance_formula'>

**Correlation** basically takes the _Covariance_ and divides it by the product of the standard deviations of each column. 
<br> This results in a number between -1 and +1. See formula below:-

<img src='https://github.com/Blackman9t/Coursera_Capstone/blob/master/correlation.png?raw=true' width=800 height=800 alt="correlation_formula">

_Covariance_ and _Correlation_ are very useful as they tell us the measure of dependence between pairs of data.

Now how do we find the _Correlation_ and _Covaraince_ of multiple columns of data

In [0]:
import numpy as np
column1 = sc.parallelize(range(100)) # Just a range of numbers from 0 - 99
column2 = sc.parallelize(range(100,200))
column3 = sc.parallelize(list(reversed(range(100))))  
# Column3 is a reversed list of values from 99 to 0. it's cast into a list cos the reversed func returns a lazy, non-serialized object
column4 = sc.parallelize(list(np.random.randint(100, size=100)))
# column4 is made of 100 random numbers from 0 to 99.

In [26]:
from pyspark.mllib.stat import Statistics
data = column1.zip(column2).zip(column3).zip(column4)\
.map(lambda a_b_c_d : (a_b_c_d[0][0][0],a_b_c_d[0][0][1],a_b_c_d[0][1],a_b_c_d[1]) )\
.map(lambda a_b_c_d : [a_b_c_d[0],a_b_c_d[1],a_b_c_d[2],a_b_c_d[3]])
data.take(10)

[[0, 100, 99, 88],
 [1, 101, 98, 9],
 [2, 102, 97, 57],
 [3, 103, 96, 27],
 [4, 104, 95, 10],
 [5, 105, 94, 45],
 [6, 106, 93, 66],
 [7, 107, 92, 34],
 [8, 108, 91, 60],
 [9, 109, 90, 10]]

Now it looks more like a relational table of rows and columns. Remember data preparation is 80% of a Data Scientist work.

In [27]:
# finally let's apply the corr function on the data
Statistics.corr(data)

array([[ 1.        ,  1.        , -1.        , -0.07228309],
       [ 1.        ,  1.        , -1.        , -0.07228309],
       [-1.        , -1.        ,  1.        ,  0.07228309],
       [-0.07228309, -0.07228309,  0.07228309,  1.        ]])

### Summary:
As we have seen, _Covariance_ explains the dependency of two columns in a data set,<br>
_Correlation_ takes the _Covariance_ and further normalises it by the standard deviations of each columns, in other to create a comparable measure between -1 and 1.<br>
The _Covariance_ matrix is a very powerful tool to display the dependencies between all columns in one single view.

### Quiz:
What is the correlation between the two lists?

1,2,3,4,5,6,7,8,9,10

7,6,5,4,5,6,7,8,9,10

Please enter at least three digits after the decimal

In [0]:
list1 = [1,2,3,4,5,6,7,8,9,10]
list2 = [7,6,5,4,5,6,7,8,9,10]

In [29]:
# First let's find the mean of both lists
list1 = sc.parallelize(list1)
list2 = sc.parallelize(list2)
mean1 = list1.sum() / float(list1.count())
mean2 = list2.sum() / float(list2.count())
print(mean1,'and', mean2)

5.5 and 6.7


In [30]:
# next let's zip both lists

zipd = list1.zip(list2)
zipd.take(5)

[(1, 7), (2, 6), (3, 5), (4, 4), (5, 5)]

In [31]:
# Now we find the covariance
cov_ = (zipd.map(lambda x_y: (x_y[0] - mean1) * (x_y[1] - mean2)).sum()) / float(zipd.count())
cov_

3.65

In [32]:
# Next we need find the standard deviation of both lists
std1 = ((list1.map(lambda x: (x - mean1)**2).sum()) / float(list1.count()))**0.5
std2 = ((list2.map(lambda x: (x - mean2)**2).sum()) / float(list2.count()))**0.5
print(std1,std2)

2.8722813232690143 1.7916472867168918


In [33]:
# So finally we find the correlation

corr = cov_ / (std1 * std2)
corr

0.7092729120837249

### Quiz
What is the covariance between the two lists?

1,2,3,4,5,6,7,8,9,10

7,6,5,4,5,6,7,8,9,10

Please enter at least three digits after the decimal